# Question 1

Multinomial logistic regression and cross validation (6 points). For this problem, you will estimate the probability that a given wine comes from a given cultivar. The data in the file strongdrink.txt (taken from the UCI Machine Learning Repository) are the results of a chemical analysis of 176 Italian wines from three known cultivars (a cultivar is a group of grapes selected for desirable characteristics that can be maintained by propagation). The chemical analysis determined the quantities of the following 13 different constituents (the last 13 variables):

In [129]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import classification_report


In [130]:
wine=pd.read_csv('data/strongdrink.txt')
print (wine.shape)
wine.head()

(176, 14)


,cultivar,alco,malic,ash,alk,magn,tot_phen,flav,nonfl_phen,proanth,color_int,hue,OD280rat,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## a) 
Use a multinomial logistic regression model of the following form with the following linear predictor $n_j$ for j = 1, 2 (the baseline class is j = 3).

$Pr(cultivar_i =j|X\beta_j)=\frac{e^{n_j}}{1+\sum{j=1}{J-1}e^{n_j}}$ for j=1,2
where $n_j = \beta_{j,0} + \beta_{j,1}alco_i + \beta_{j,2}malic_i + \beta_{j,3}totphen_i + \beta_{j,4}colorint_i$

Estimate the model on a 75% sample training set using the following command. Report your two sets of estimated coefficients and intercepts for j = 1 and j = 2 (not the coefficients for j = 3). Report your error rates (1 - precision) on the test set using the code below. Which category(ies) of cultivar is the model best at predicting? Is (are) the most accurately predicted category(ies) the one(s) with the most observations? Report the MSE from the test set.

The error rate for Cultivars 1, 2, and 3 is 0.13, 0.0 and 0.0, respectively. The most accurately predicted category of cultivar is the #3, which is the category with the lowest number of observations. The MSE of the test set is 0.04545

In [133]:
#Split data
X = wine[['alco','malic','tot_phen','color_int']]
y = wine['cultivar']


(array([1, 2, 3]), array([13, 21, 10]))

In [134]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.25,
       random_state=20)

#Count number of each cultivar in test set
np.unique(y_test, return_counts=True)

(array([1, 2, 3]), array([13, 21, 10]))

In [97]:
LogReg = LogisticRegression(random_state=0, multi_class='multinomial',solver='newton-cg')
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)


print('The coefficients were Cultivar 1 are:', LogReg.coef_[1])
print('The coefficients were Cultivar 2 are:', LogReg.coef_[2])

The coefficients were Cultivar 1 are: [-1.46798523 -0.33305092  0.66400603 -0.92270882]
The coefficients were Cultivar 2 are: [-0.2324475   0.59866064 -1.8879004   0.89996106]


In [98]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[13  0  0]
 [ 2 19  0]
 [ 0  0 10]]


In [102]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.87      1.00      0.93        13
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00        10

   micro avg       0.95      0.95      0.95        44
   macro avg       0.96      0.97      0.96        44
weighted avg       0.96      0.95      0.96        44



In [103]:
print('Cultivar 1 error rate = 0.13')
print('Cultivar 2 error rate = 0.0')
print('Cultivar 3 error rate = 0.0')

Cultivar 1 error rate = 0.13
Cultivar 2 error rate = 0.0
Cultivar 3 error rate = 0.0


In [104]:
mse_test = (((y_test-y_pred)**2).sum())/y_pred.shape[0]
print('The MSE from the test is:', mse_test)

The MSE from the test is: 0.045454545454545456
